In [1]:
pip install bs4

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests, os
from bs4 import BeautifulSoup

def scrape_firs(district_id, fir_start, fir_end, output_dir):
    url = "https://ksp.karnataka.gov.in/"
    stations_url = f"https://ksp.karnataka.gov.in/myform/ajax/{district_id}?unit_name={district_id}"
    stations = requests.get(stations_url).json()
    ps_ids = [obj["station_id"] for obj in stations]

    headers = {
        "Content-Type": "application/x-www-form-urlencoded",
        "User-Agent": "Mozilla/5.0",
    }
    
    data = {
        "district_id": district_id,
        "knen": "en",
        "random_captcha": "LOL",
        "captcha": "LOL",
    }

    for ps_id in ps_ids:
        for num in range(fir_start, fir_end + 1):
            data.update({"ps_id": ps_id, "fir_num": f"{num:04}"})
            response = requests.post("https://ksp.karnataka.gov.in/fir_search_new_s.php", headers=headers, data=data)

            if "FIR Not Found!" in response.text:
                print(f"FIR not found for station {ps_id} and FIR number {data['fir_num']}")
                continue

            soup = BeautifulSoup(response.content, "html.parser")
            pdf_url = url + soup.a["href"]
            ps_path = os.path.join(output_dir, str(ps_id))
            os.makedirs(ps_path, exist_ok=True)

            with open(f"{ps_path}/fir_{data['fir_num']}.pdf", "wb") as f:
                f.write(requests.get(pdf_url).content)
